<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-01-07 13:46:15
-------------------
qualified stocks: 89
with latest results: 84
still star stocks: 44
-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  0.00
Current:  1.20 C
-------------------
Today PnL: 91.92 K (0.77%)
Current PnL: -9.10 L (-7.78%)
CY Booked + Current PnL: 2.40 L (2.05%)
-------------------
Total profit:  4.83 L
Total loss:  -13.93 L
-------------------
Total Booked + Current PnL: 15.51 L (14.92%)
Total Booked PnL: 24.61 L (23.68%)
Curr Year Booked PnL: 11.50 L (9.61%)
Prev Year Booked PnL: 13.11 L (12.62%)
Est FTT:  1.87 C
Est FTT PnL: 67.48 L (56.39%)
-------------------
Est LTT:  2.48 C
Est LTT PnL: 1.28 C (107.29%)
Deployed:  1.04 C
Current:  1.20 C
CAGR/XIRR %: 12.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost       FTT  Dev%_PE Conviction    RSP   Current  \
12  BANKBARODA   195.74    283.00   -26.20          M  13.48   92086.0   
15   BAYERCROP  4346.57   6836.59    20.10          H  35.96   89074.0   
48  LALPATHLAB  2630.06   3690.00    -9.62          H  76.40  137368.0   
72         SIS   477.00    528.00    65.72          M  21.35   59840.0   
20        CERA  9059.90  10420.45    -4.52          H  50.56   73844.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
12      14769.0  19697.0        2.27         19.10  21.39  44.58  19.0   
15      19529.0  20309.0        1.13         28.08  22.80  57.29  21.0   
48      21645.0  24987.0        4.05         18.70  18.19  40.30  19.0   
72     -21250.0  29920.0       -0.69        -26.21  50.00  10.69 -21.0   
20     -16755.0  30357.0        1.32        -18.49  41.11  15.02 -51.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
12     0.75        0.77     7.75       XR      NTT       BANKS  
15     0.96        0.74    14.04     X40N      ATH   CHEMICALS  
48     0.87        1.15    56.16     X40N      NTT  HEALTHCARE  
72    -0.71        0.50     0.00     X40N      NTT        MISC  
20    -0.55        0.62    12.11     X40N      ATH    CERAMICS

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
70         SFL  1034.30  1287.0    35.61          M  98.88  252284.0   
22   COFFEEDAY    59.14    80.0  -131.17          L  96.63   55373.0   
27  EASEMYTRIP    18.68    26.4    -4.74          M  53.93  144310.0   
48  LALPATHLAB  2630.06  3690.0    -9.62          H  76.40  137368.0   
81    TITAGARH  1117.46  1548.0     1.99          M  39.33  183538.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
70        -85.0  61734.0        5.51         -0.03   24.47  24.43  -7.0   
22     -58176.0  98226.0        4.99        -51.23  177.39  35.27   NaN   
27     -22091.0  90858.0        4.45        -13.28   62.96  41.33 -93.0   
48      21645.0  24987.0        4.05         18.70   18.19  40.30  19.0   
81       1392.0  68790.0        3.50          0.76   37.48  38.53  14.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy     Category  
70    -0.00        2.11    32.55     XY24      NTT         MISC  
22    -0.59        0.46    27.50       XR      NTT       HOTELS  
27    -0.24        1.21    12.19     XY24      NTT       TRAVEL  
48     0.87        1.15    56.16     X40N      NTT   HEALTHCARE  
81     0.02        1.53    36.75     XY24      NTT  ENGINEERING

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
45  JPPOWER    18.73    26.2   -64.86          L  32.02  127075.0   
29    GICRE   335.06   583.0    10.41          M  97.75  190117.0   
56     NHPC    82.08   115.0   183.44          M  33.71  136186.0   
19   CAMPUS   294.86   393.0    -2.42          M  78.65  170814.0   
30    GLAND  1463.94  3442.0     9.41          H  89.89  100251.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%   XIRR  \
45     -14767.0  71340.0       -5.04        -10.41  56.14   39.88   85.0   
29      43696.0  64659.0       -4.95         29.84  34.01   74.00  137.0   
56      -4827.0  61379.0       -4.60         -3.42  45.07   40.11  -85.0   
19       3039.0  52799.0       -2.48          1.81  30.91   33.28   18.0   
30      21198.0  85614.0       -1.81         26.82  85.40  135.12   29.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
45    -0.21        1.06    12.24     XY24      NTT      POWER  
29     0.68        1.59    46.96     XY24      BTT  INSURANCE  
56    -0.08        1.14    17.07     XY24      NTT      POWER  
19     0.06        1.43    39.95     XY24      NTT   FOOTWEAR  
30     0.25        0.84    15.61       XR      NTT     PHARMA

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
51      LUXIND  1448.19  2442.00    44.24          M  48.31  182101.0   
15   BAYERCROP  4346.57  6836.59    20.10          H  35.96   89074.0   
54  METROPOLIS  1399.27  3102.00    18.62          M  42.70  136968.0   
83  TTKPRESTIG   769.29  1149.24    18.61          H  26.97  106117.0   
75       SUNTV   536.47  1092.00    12.51          M  12.36  122308.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  XIRR  \
51      44523.0  49896.0        1.25         32.36  27.40   68.62  22.0   
15      19529.0  20309.0        1.13         28.08  22.80   57.29  21.0   
54      43217.0  70867.0        0.27         46.10  51.74  121.69  33.0   
83       5340.0  44431.0        0.04          5.30  41.87   49.39   6.0   
75      24134.0  77531.0       -0.04         24.58  63.39  103.55  20.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy       Category  
51     0.89        1.52    78.58     X40N      NTT       TEXTILES  
15     0.96        0.74    14.04     X40N      ATH      CHEMICALS  
54     0.61        1.14    35.94       XR      NTT     HEALTHCARE  
83     0.12        0.89    21.89     X40N      ATH       DURABLES  
75     0.31        1.02    18.37     X40N      BTT  ENTERTAINMENT

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
38         IEX   133.39   219.0    -5.37          M  55.06  195299.0   
12  BANKBARODA   195.74   283.0   -26.20          M  13.48   92086.0   
54  METROPOLIS  1399.27  3102.0    18.62          M  42.70  136968.0   
26       DMART  3504.64  5112.0   -23.63          H  86.52  143891.0   
30       GLAND  1463.94  3442.0     9.41          H  89.89  100251.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  XIRR  \
38      46036.0  49762.0        0.52         30.84  25.48   64.18  28.0   
12      14769.0  19697.0        2.27         19.10  21.39   44.58  19.0   
54      43217.0  70867.0        0.27         46.10  51.74  121.69  33.0   
26      14219.0  45254.0       -1.16         10.97  31.45   45.86  35.0   
30      21198.0  85614.0       -1.81         26.82  85.40  135.12  29.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
38     0.93        1.63    32.41       XR      NTT        MISC  
12     0.75        0.77     7.75       XR      NTT       BANKS  
54     0.61        1.14    35.94       XR      NTT  HEALTHCARE  
26     0.31        1.20    14.10       AR      NTT        FMCG  
30     0.25        0.84    15.61       XR      NTT      PHARMA

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
48  LALPATHLAB  2630.06  3690.0    -9.62          H  76.40  137368.0   
31  GREENPANEL   390.01   537.0    84.81          M  92.13  161894.0   
19      CAMPUS   294.86   393.0    -2.42          M  78.65  170814.0   
51      LUXIND  1448.19  2442.0    44.24          M  48.31  182101.0   
29       GICRE   335.06   583.0    10.41          M  97.75  190117.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
48      21645.0  24987.0        4.05         18.70  18.19  40.30   19.0   
31     -10880.0  75993.0        2.70         -6.30  46.94  37.69  -14.0   
19       3039.0  52799.0       -2.48          1.81  30.91  33.28   18.0   
51      44523.0  49896.0        1.25         32.36  27.40  68.62   22.0   
29      43696.0  64659.0       -4.95         29.84  34.01  74.00  137.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
48     0.87        1.15    56.16     X40N      NTT  HEALTHCARE  
31    -0.14        1.35    27.67     XY24      NTT        MISC  
19     0.06        1.43    39.95     XY24      NTT    FOOTWEAR  
51     0.89        1.52    78.58     X40N      NTT    TEXTILES  
29     0.68        1.59    46.96     XY24      BTT   INSURANCE

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
21       CLEAN  1389.74  2137.0    -5.20          H  100.00  120015.0   
70         SFL  1034.30  1287.0    35.61          M   98.88  252284.0   
29       GICRE   335.06   583.0    10.41          M   97.75  190117.0   
22   COFFEEDAY    59.14    80.0  -131.17          L   96.63   55373.0   
10  BAJFINANCE  7055.29  9150.0   -27.03          H   95.51  228470.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%   XIRR  \
21       6056.0  55219.0        1.30          5.31   46.01  53.77    2.0   
70        -85.0  61734.0        5.51         -0.03   24.47  24.43   -7.0   
29      43696.0  64659.0       -4.95         29.84   34.01  74.00  137.0   
22     -58176.0  98226.0        4.99        -51.23  177.39  35.27    NaN   
10       9756.0  55176.0        0.28          4.46   24.15  29.69   12.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
21     0.11        1.00    14.72       XR      NTT  CHEMICALS  
70    -0.00        2.11    32.55     XY24      NTT       MISC  
29     0.68        1.59    46.96     XY24      BTT  INSURANCE  
22    -0.59        0.46    27.50       XR      NTT     HOTELS  
10     0.18        1.91    17.36      X40      BTT    FINANCE

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.44
1     25       40.76
2     50       70.87

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    40.60
X40     21.09
X40N    14.92
XR      14.37
AR       6.16
X200     1.45
SR       1.38
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
M    57.36
H    39.20
L     3.41
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
BANKS               14.22        -12.08   65.14
FINANCE             12.38        -10.29   56.73
IT                   8.69         -7.11   60.94
FMCG                 7.79         -1.70   39.76
MISC                 7.57         -0.02   37.09
PAINTS               5.84        -17.52   52.30
CHEMICALS            5.65         -6.88   74.39
HEALTHCARE           4.99          8.68   37.58
INSURANCE            4.27          5.21   45.93
AUTO                 3.31         -4.00   49.73
FOOTWEAR             3.09        -11.69   48.59
TRAVEL               2.22        -15.76   63.02
TEXTILES             2.20         12.73   33.11
POWER                2.20         -7.44   50.41
ENTERTAINMENT        2.11         -3.53   76.29
CERAMICS             2.04        -16.73   64.22
STEEL                1.61        -16.69  103.66
JEWELLERY            1.55        -55.88  105.18
ENGINEERING          1.53          0.76   37.48
REFINERIES           1.33         -3.49   28.57
CEMENT               1.23        -28.19   94.14
ELECTRICAL           1.07          0.33   36.76
DURABLES             0.89          5.03   41.87
AC                   0.89        -20.61   87.19
PHARMA               0.84         21.14   85.40
HOTELS               0.46       -105.06  177.39

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      2796563.0     30
XR        1294477.0     18
X40N       956320.0     17
X40        860239.0     14
AR         615911.0      7
SR         179672.0      2
X200        44808.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
M           4628604.0     53
H           1734984.0     31
L            384402.0      5

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M          XY24      2443351.0     25
           XR         893381.0     13
H          X40        804656.0     13
M          X40N       573842.0      8
           AR         482775.0      4
H          X40N       382478.0      9
           XY24       281872.0      4
L          XR         260263.0      3
M          SR         179672.0      2
H          XR         140833.0      2
           AR          80337.0      2
L          XY24        71340.0      1
M          X40         55583.0      1
L          AR          52799.0      1
H          X200        44808.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
